In [1]:
# 데이터 불러오기
from dotenv import load_dotenv
import pandas as pd
import os 
import numpy as np
from sqlalchemy import create_engine


In [2]:
# .env 파일에서 환경 부르기
load_dotenv()

# 환경 변수 사용
database = os.getenv("CON_STR")


# csv 파일 경로
csv_file_path = os.getenv("CSV_FILE_PATH")
print(csv_file_path)

C:\Users\schoi43\OneDrive - Philip Morris International\Documents\BGFR_PMI_BackData\BGFR_PMI_202306.csv


In [3]:
print(database)

engine = create_engine(database)


mssql+pyodbc://@CU_DB?Trusted_Connection=yes;


In [119]:
# -- Arbor_sourcing_M1 작업 
data['company'] = data['company'].apply(lambda x: ','.join(dict.fromkeys(map(str.strip, x.split(',')))))

data['FLAVOR'] = data['FLAVOR'].apply(lambda x: ','.join(dict.fromkeys(map(str.strip, x.split(',')))))

# data2.to_csv('prac.csv', index=False)

In [142]:
# -- Arbor_sourcing_M1
# 직전 3개월 데이터 구매 이력 소싱 - raw data
sql = ''' 
	select 
		t.YYYYMM, a.id, t.gender, t.age, 
        CASE 
		    WHEN SUM(CASE WHEN b.cigatype = 'CC' THEN 1 ELSE 0 END) > 0 
			 AND SUM(CASE WHEN b.cigatype = 'HnB' THEN 1 ELSE 0 END) > 0 
			THEN 'Mixed' 
			ELSE MAX(b.cigatype)  -- CC 또는 HnB가 없을 경우 가장 큰 값을 사용
		END AS cigatype,
		company,
        ProductFamilyCode,
        productSubFamilyCode,
		New_FLAVORSEG,
        New_TARSEGMENTAT,
		engname,
	    sum(a.pack_qty) qty
	from  cx.agg_TEREA_SKU_Sourcing t
		join cx.fct_K7_Monthly a on t.id = a.id 
			and a.YYYYMM BETWEEN CONVERT(NVARCHAR(6), DATEADD(MONTH, -3, t.YYYYMM+'01'), 112)
							 AND CONVERT(NVARCHAR(6), DATEADD(MONTH, -1, t.YYYYMM+'01'), 112)	
		join cx.product_master_temp b on a.product_code = b.PROD_ID and CIGADEVICE =  'CIGARETTES' AND  cigatype != 'CSV' AND 4 < LEN(a.id)
	where t.YYYYMM='202403'	
	group by 
		t.YYYYMM, a.id, t.gender, t.age,
		company,
        ProductFamilyCode,
        productSubFamilyCode,
		New_FLAVORSEG,
        New_TARSEGMENTAT,
		engname
'''

data = pd.read_sql_query(sql=sql, con=engine)


In [171]:
data

,YYYYMM,id,gender,age,cigatype,company,ProductFamilyCode,productSubFamilyCode,New_FLAVORSEG,New_TARSEGMENTAT,engname,qty,CC_taste,HnB_taste
0,202403,006428C6FFBB774A4EC6356CE797E9E9B8CD3AC9FD3591...,남,20대,HnB,BAT,GLO,NEO,New Taste,None,NEO JUICY BOOST,1.0,HnB New Taste,NEO New Taste
1,202403,006428C6FFBB774A4EC6356CE797E9E9B8CD3AC9FD3591...,남,20대,HnB,PMK,IQOS,TEREA,Fresh,None,TEREA BLUE,3.0,HnB Fresh,TEREA Fresh
2,202403,018C696532DAFAE92FE2FEE1F3A39676AC8A125BB87FAA...,남,20대,HnB,KTG,LIL,MIIX,New Taste,None,MIIX ICE DOUBLE,2.0,HnB New Taste,MIIX New Taste
3,202403,018C696532DAFAE92FE2FEE1F3A39676AC8A125BB87FAA...,남,20대,HnB,PMK,IQOS,TEREA,New Taste,None,TEREA SUMMER WAVE,11.0,HnB New Taste,TEREA New Taste
4,202403,021908C063110E1783B46C79E075D40CB2CA734CE613F0...,남,40대,HnB,PMK,IQOS,TEREA,New Taste,None,TEREA SUN PEARL,1.0,HnB New Taste,TEREA New Taste
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2195,202403,FFCA66B0C9AD8BBE0B95471AECD627D6D52E3482EE0FCD...,남,20대,CC,BAT,DUNHILL,DUNHILLKS,Fresh,1MG,DUNHILL FROST,1.0,CC Fresh,NaN
2196,202403,FFCA66B0C9AD8BBE0B95471AECD627D6D52E3482EE0FCD...,남,20대,CC,KTG,ESSE,ESSEC,New Taste,1MG,ESSE CHANGE HIMALAYA,2.0,CC New Taste,NaN
2197,202403,FFCA66B0C9AD8BBE0B95471AECD627D6D52E3482EE0FCD...,남,20대,HnB,PMK,IQOS,TEREA,New Taste,None,TEREA OASIS PEARL,2.0,HnB New Taste,TEREA New Taste
2198,202403,FFCA66B0C9AD8BBE0B95471AECD627D6D52E3482EE0FCD...,남,20대,HnB,PMK,IQOS,TEREA,New Taste,None,TEREA SUN PEARL,4.0,HnB New Taste,TEREA New Taste


In [160]:
pivot_table = data.pivot_table(
    index=['YYYYMM', 'id', 'gender', 'age'] ,           # 행 인덱스
    columns=['company'],
    values=['qty'],
    aggfunc='sum'            
)

pivot_table 

qty  \
company                                                              BAT            
YYYYMM id                                                 gender age                
202403 006428C6FFBB774A4EC6356CE797E9E9B8CD3AC9FD35910... 남      20대          1.0   
       018C696532DAFAE92FE2FEE1F3A39676AC8A125BB87FAA9... 남      20대          NaN   
       021908C063110E1783B46C79E075D40CB2CA734CE613F07... 남      40대          NaN   
       02633A5CED7C5FE7E2F9FB6A5F6198383BFF6159C1B70C4... 여      40대          NaN   
       0265F2F8BC605B1EE0A9670903BEDE162DC0E6DC742B761... 남      40대          NaN   
...                                                                           ...   
       FF42DEB477AB28D91C8D94DB43E988E380867F2494EE59D... 남      40대          2.0   
       FF8F1FFB4BD5D09BFF62A4A07C749E7B956AA3FC88757B8... 여      60대         29.0   
       FFC79A4B73F8EC9BA2D1FF586F951173302BEEE799AED98... 남      40대          NaN   
       FFCA66B0C9AD8BBE0B95471AECD627D6D52E3482EE0FCD3... 남      20대          1.0   
       FFDEE2B8A21B7DA963852B0AFECA0D3A687DBF295F4F726... 남      20대          NaN   

                                                                                   \
company                                                              JTI            
YYYYMM id                                                 gender age                
202403 006428C6FFBB774A4EC6356CE797E9E9B8CD3AC9FD35910... 남      20대          NaN   
       018C696532DAFAE92FE2FEE1F3A39676AC8A125BB87FAA9... 남      20대          NaN   
       021908C063110E1783B46C79E075D40CB2CA734CE613F07... 남      40대          NaN   
       02633A5CED7C5FE7E2F9FB6A5F6198383BFF6159C1B70C4... 여      40대          NaN   
       0265F2F8BC605B1EE0A9670903BEDE162DC0E6DC742B761... 남      40대          NaN   
...                                                                           ...   
       FF42DEB477AB28D91C8D94DB43E988E380867F2494EE59D... 남      40대          NaN   
       FF8F1FFB4BD5D09BFF62A4A07C749E7B956AA3FC88757B8... 여      60대          NaN   
       FFC79A4B73F8EC9BA2D1FF586F951173302BEEE799AED98... 남      40대          NaN   
       FFCA66B0C9AD8BBE0B95471AECD627D6D52E3482EE0FCD3... 남      20대          NaN   
       FFDEE2B8A21B7DA963852B0AFECA0D3A687DBF295F4F726... 남      20대          NaN   

                                                                                   \
company                                                              KTG            
YYYYMM id                                                 gender age                
202403 006428C6FFBB774A4EC6356CE797E9E9B8CD3AC9FD35910... 남      20대          NaN   
       018C696532DAFAE92FE2FEE1F3A39676AC8A125BB87FAA9... 남      20대          2.0   
       021908C063110E1783B46C79E075D40CB2CA734CE613F07... 남      40대          NaN   
       02633A5CED7C5FE7E2F9FB6A5F6198383BFF6159C1B70C4... 여      40대         37.0   
       0265F2F8BC605B1EE0A9670903BEDE162DC0E6DC742B761... 남      40대          NaN   
...                                                                           ...   
       FF42DEB477AB28D91C8D94DB43E988E380867F2494EE59D... 남      40대          NaN   
       FF8F1FFB4BD5D09BFF62A4A07C749E7B956AA3FC88757B8... 여      60대         83.0   
       FFC79A4B73F8EC9BA2D1FF586F951173302BEEE799AED98... 남      40대          NaN   
       FFCA66B0C9AD8BBE0B95471AECD627D6D52E3482EE0FCD3... 남      20대          2.0   
       FFDEE2B8A21B7DA963852B0AFECA0D3A687DBF295F4F726... 남      20대          NaN   

                                                                                   
company                                                              PMK           
YYYYMM id                                                 gender age               
202403 006428C6FFBB774A4EC6356CE797E9E9B8CD3AC9FD35910... 남      20대          3.0  
       018C696532DAFAE92FE2FEE1F3A39676AC8A125BB87FAA9... 남      20대         11.0  
       021908C063110E1783B46C79E075D40CB2CA734CE613F07... 남      40대          1.0  
 

In [161]:
data['CC_taste'] = data['cigatype'] + ' ' + data['New_FLAVORSEG']

pivot_table2 = data.pivot_table(
    index=['YYYYMM', 'id', 'gender', 'age'] ,
    columns=['CC_taste'],
    values=['qty'],  
    aggfunc='sum'           
)

pivot_table2

qty  \
CC_taste                                                             CC Fresh   
YYYYMM id                                                 gender age            
202403 006428C6FFBB774A4EC6356CE797E9E9B8CD3AC9FD35910... 남      20대      NaN   
       018C696532DAFAE92FE2FEE1F3A39676AC8A125BB87FAA9... 남      20대      NaN   
       021908C063110E1783B46C79E075D40CB2CA734CE613F07... 남      40대      NaN   
       02633A5CED7C5FE7E2F9FB6A5F6198383BFF6159C1B70C4... 여      40대      NaN   
       0265F2F8BC605B1EE0A9670903BEDE162DC0E6DC742B761... 남      40대      NaN   
...                                                                       ...   
       FF42DEB477AB28D91C8D94DB43E988E380867F2494EE59D... 남      40대      NaN   
       FF8F1FFB4BD5D09BFF62A4A07C749E7B956AA3FC88757B8... 여      60대      NaN   
       FFC79A4B73F8EC9BA2D1FF586F951173302BEEE799AED98... 남      40대      NaN   
       FFCA66B0C9AD8BBE0B95471AECD627D6D52E3482EE0FCD3... 남      20대      1.0   
       FFDEE2B8A21B7DA963852B0AFECA0D3A687DBF295F4F726... 남      20대      3.0   

                                                                                   \
CC_taste                                                             CC New Taste   
YYYYMM id                                                 gender age                
202403 006428C6FFBB774A4EC6356CE797E9E9B8CD3AC9FD35910... 남      20대          NaN   
       018C696532DAFAE92FE2FEE1F3A39676AC8A125BB87FAA9... 남      20대          NaN   
       021908C063110E1783B46C79E075D40CB2CA734CE613F07... 남      40대          NaN   
       02633A5CED7C5FE7E2F9FB6A5F6198383BFF6159C1B70C4... 여      40대         35.0   
       0265F2F8BC605B1EE0A9670903BEDE162DC0E6DC742B761... 남      40대          NaN   
...                                                                           ...   
       FF42DEB477AB28D91C8D94DB43E988E380867F2494EE59D... 남      40대          NaN   
       FF8F1FFB4BD5D09BFF62A4A07C749E7B956AA3FC88757B8... 여      60대         11.0   
       FFC79A4B73F8EC9BA2D1FF586F951173302BEEE799AED98... 남      40대          NaN   
       FFCA66B0C9AD8BBE0B95471AECD627D6D52E3482EE0FCD3... 남      20대          2.0   
       FFDEE2B8A21B7DA963852B0AFECA0D3A687DBF295F4F726... 남      20대          NaN   

                                                                                 \
CC_taste                                                             CC Regular   
YYYYMM id                                                 gender age              
202403 006428C6FFBB774A4EC6356CE797E9E9B8CD3AC9FD35910... 남      20대        NaN   
       018C696532DAFAE92FE2FEE1F3A39676AC8A125BB87FAA9... 남      20대        NaN   
       021908C063110E1783B46C79E075D40CB2CA734CE613F07... 남      40대        NaN   
       02633A5CED7C5FE7E2F9FB6A5F6198383BFF6159C1B70C4... 여      40대        1.0   
       0265F2F8BC605B1EE0A9670903BEDE162DC0E6DC742B761... 남      40대        NaN   
...                                                                         ...   
       FF42DEB477AB28D91C8D94DB43E988E380867F2494EE59D... 남      40대        2.0   
       FF8F1FFB4BD5D09BFF62A4A07C749E7B956AA3FC88757B8... 여      60대       31.0   
       FFC79A4B73F8EC9BA2D1FF586F951173302BEEE799AED98... 남      40대        NaN   
       FFCA66B0C9AD8BBE0B95471AECD627D6D52E3482EE0FCD3... 남      20대        NaN   
       FFDEE2B8A21B7DA963852B0AFECA0D3A687DBF295F4F726... 남      20대        NaN   

                                                                                \
CC_taste                                                             HnB Fresh   
YYYYMM id                                                 gender age             
202403 006428C6FFBB774A4EC6356CE797E9E9B8CD3AC9FD35910... 남      20대       3.0   
       018C696532DAFAE92FE2FEE1F3A39676AC8A125BB87FAA9... 남      20대       NaN   
       021908C063110E1783B46C79E075D40CB2CA734CE613F07... 남      40대       NaN   
       02633A5CED7C5FE7E2F9FB6A5F6198383BFF6159C1B70C4... 여      40대       NaN   
       0265

In [149]:

result =data[data['productSubFamilyCode'].isin(['AIIM', 'FIIT', 'HEETS', 'MIIX', 'NEO', 'NEOSTICKS', 'TEREA'])]

data['HnB_taste'] = result['productSubFamilyCode'] + ' ' + data['New_FLAVORSEG']

pivot_table3 = data.pivot_table(
    index=['YYYYMM', 'id', 'gender', 'age'] ,           # 행 인덱스
    columns=['HnB_taste'],
    values=['qty'],  # 피벗할 값
    aggfunc='sum'            # 집계 함수 (여기서는 평균)
)

pivot_table3

qty  \
HnB_taste                                                            AIIM Fresh   
YYYYMM id                                                 gender age              
202403 006428C6FFBB774A4EC6356CE797E9E9B8CD3AC9FD35910... 남      20대        NaN   
       018C696532DAFAE92FE2FEE1F3A39676AC8A125BB87FAA9... 남      20대        NaN   
       021908C063110E1783B46C79E075D40CB2CA734CE613F07... 남      40대        NaN   
       02633A5CED7C5FE7E2F9FB6A5F6198383BFF6159C1B70C4... 여      40대        NaN   
       0265F2F8BC605B1EE0A9670903BEDE162DC0E6DC742B761... 남      40대        NaN   
...                                                                         ...   
       FE51C541A50F8F4E721DC78C28FF86EC76D9E324D479BB5... 남      30대        NaN   
       FF1635DD2593732635DBA615CDCDB68FB8763D37D728D98... 남      40대        NaN   
       FF8F1FFB4BD5D09BFF62A4A07C749E7B956AA3FC88757B8... 여      60대        NaN   
       FFC79A4B73F8EC9BA2D1FF586F951173302BEEE799AED98... 남      40대        NaN   
       FFCA66B0C9AD8BBE0B95471AECD627D6D52E3482EE0FCD3... 남      20대        NaN   

                                                                                     \
HnB_taste                                                            AIIM New Taste   
YYYYMM id                                                 gender age                  
202403 006428C6FFBB774A4EC6356CE797E9E9B8CD3AC9FD35910... 남      20대            NaN   
       018C696532DAFAE92FE2FEE1F3A39676AC8A125BB87FAA9... 남      20대            NaN   
       021908C063110E1783B46C79E075D40CB2CA734CE613F07... 남      40대            NaN   
       02633A5CED7C5FE7E2F9FB6A5F6198383BFF6159C1B70C4... 여      40대            NaN   
       0265F2F8BC605B1EE0A9670903BEDE162DC0E6DC742B761... 남      40대            NaN   
...                                                                             ...   
       FE51C541A50F8F4E721DC78C28FF86EC76D9E324D479BB5... 남      30대            NaN   
       FF1635DD2593732635DBA615CDCDB68FB8763D37D728D98... 남      40대            NaN   
       FF8F1FFB4BD5D09BFF62A4A07C749E7B956AA3FC88757B8... 여      60대            NaN   
       FFC79A4B73F8EC9BA2D1FF586F951173302BEEE799AED98... 남      40대            NaN   
       FFCA66B0C9AD8BBE0B95471AECD627D6D52E3482EE0FCD3... 남      20대            NaN   

                                                                                   \
HnB_taste                                                            AIIM Regular   
YYYYMM id                                                 gender age                
202403 006428C6FFBB774A4EC6356CE797E9E9B8CD3AC9FD35910... 남      20대          NaN   
       018C696532DAFAE92FE2FEE1F3A39676AC8A125BB87FAA9... 남      20대          NaN   
       021908C063110E1783B46C79E075D40CB2CA734CE613F07... 남      40대          NaN   
       02633A5CED7C5FE7E2F9FB6A5F6198383BFF6159C1B70C4... 여      40대          NaN   
       0265F2F8BC605B1EE0A9670903BEDE162DC0E6DC742B761... 남      40대          NaN   
...                                                                           ...   
       FE51C541A50F8F4E721DC78C28FF86EC76D9E324D479BB5... 남      30대          NaN   
       FF1635DD2593732635DBA615CDCDB68FB8763D37D728D98... 남      40대          NaN   
       FF8F1FFB4BD5D09BFF62A4A07C749E7B956AA3FC88757B8... 여      60대          NaN   
       FFC79A4B73F8EC9BA2D1FF586F951173302BEEE799AED98... 남      40대          NaN   
       FFCA66B0C9AD8BBE0B95471AECD627D6D52E3482EE0FCD3... 남      20대          NaN   

                                                                                 \
HnB_taste                                                            FIIT Fresh   
YYYYMM id                                                 gender age              
202403 006428C6FFBB774A4EC6356CE797E9E9B8CD3AC9FD35910... 남      20대        NaN   
       018C696532DAFAE92FE2FEE1F3A39676AC8A125BB87FAA9... 남      20대        NaN   
       021908C063110E1783B46C79E075D40CB2CA734CE613F07... 남      40대        NaN   
     

In [154]:
filtered_data = data[data['ProductFamilyCode'] == 'IQOS'][['engname']]
filtered_data['engname'].unique()

array(['TEREA BLUE', 'TEREA SUMMER WAVE', 'TEREA SUN PEARL',
       'TEREA PURPLE WAVE', 'TEREA TEAK', 'TEREA RUSSET', 'TEREA SILVER',
       'TEREA OASIS PEARL', 'TEREA BLACK PURPLE', 'TEREA YUGEN',
       'TEREA GREEN', 'TEREA AMBER', 'TEREA BLACK YELLOW',
       'HEETS AMBER LABEL', 'HEETS PURPLE LABEL', 'TEREA BLACK GREEN',
       'HEETS BLACK PURPLE SELECTION', 'HEETS YUGEN', 'TEREA GREEN ZING',
       'HEETS SILVER LABEL', 'HEETS GREEN LABEL', 'HEETS BLUE LABEL',
       'HEETS SATIN WAVE', 'HEETS BLACK GREEN SELECTION',
       'HEETS SUMMER BREEZE'], dtype=object)

In [158]:
filtered_data = data[data['ProductFamilyCode'] == 'IQOS']

pivot_table4 = filtered_data.pivot_table(
    index=['YYYYMM', 'id', 'gender', 'age'] ,           # 행 인덱스
    columns=['engname'],
    values=['qty'],  # 피벗할 값
    aggfunc='sum'            # 집계 함수 (여기서는 평균)
)

pivot_table4

qty  \
engname                                                              HEETS AMBER LABEL   
YYYYMM id                                                 gender age                     
202403 006428C6FFBB774A4EC6356CE797E9E9B8CD3AC9FD35910... 남      20대               NaN   
       018C696532DAFAE92FE2FEE1F3A39676AC8A125BB87FAA9... 남      20대               NaN   
       021908C063110E1783B46C79E075D40CB2CA734CE613F07... 남      40대               NaN   
       02633A5CED7C5FE7E2F9FB6A5F6198383BFF6159C1B70C4... 여      40대               NaN   
       0265F2F8BC605B1EE0A9670903BEDE162DC0E6DC742B761... 남      40대               NaN   
...                                                                                ...   
       FE51C541A50F8F4E721DC78C28FF86EC76D9E324D479BB5... 남      30대               NaN   
       FF1635DD2593732635DBA615CDCDB68FB8763D37D728D98... 남      40대               NaN   
       FF8F1FFB4BD5D09BFF62A4A07C749E7B956AA3FC88757B8... 여      60대               NaN   
       FFC79A4B73F8EC9BA2D1FF586F951173302BEEE799AED98... 남      40대               NaN   
       FFCA66B0C9AD8BBE0B95471AECD627D6D52E3482EE0FCD3... 남      20대               NaN   

                                                                                                  \
engname                                                              HEETS BLACK GREEN SELECTION   
YYYYMM id                                                 gender age                               
202403 006428C6FFBB774A4EC6356CE797E9E9B8CD3AC9FD35910... 남      20대                         NaN   
       018C696532DAFAE92FE2FEE1F3A39676AC8A125BB87FAA9... 남      20대                         NaN   
       021908C063110E1783B46C79E075D40CB2CA734CE613F07... 남      40대                         NaN   
       02633A5CED7C5FE7E2F9FB6A5F6198383BFF6159C1B70C4... 여      40대                         NaN   
       0265F2F8BC605B1EE0A9670903BEDE162DC0E6DC742B761... 남      40대                         NaN   
...                                                                                          ...   
       FE51C541A50F8F4E721DC78C28FF86EC76D9E324D479BB5... 남      30대                         NaN   
       FF1635DD2593732635DBA615CDCDB68FB8763D37D728D98... 남      40대                         NaN   
       FF8F1FFB4BD5D09BFF62A4A07C749E7B956AA3FC88757B8... 여      60대                         NaN   
       FFC79A4B73F8EC9BA2D1FF586F951173302BEEE799AED98... 남      40대                         NaN   
       FFCA66B0C9AD8BBE0B95471AECD627D6D52E3482EE0FCD3... 남      20대                         NaN   

                                                                                                   \
engname                                                              HEETS BLACK PURPLE SELECTION   
YYYYMM id                                                 gender age                                
202403 006428C6FFBB774A4EC6356CE797E9E9B8CD3AC9FD35910... 남      20대                          NaN   
       018C696532DAFAE92FE2FEE1F3A39676AC8A125BB87FAA9... 남      20대                          NaN   
       021908C063110E1783B46C79E075D40CB2CA734CE613F07... 남      40대                          NaN   
       02633A5CED7C5FE7E2F9FB6A5F6198383BFF6159C1B70C4... 여      40대                          NaN   
       0265F2F8BC605B1EE0A9670903BEDE162DC0E6DC742B761... 남      40대                          NaN   
...                                                                                           ...   
       FE51C541A50F8F4E721DC78C28FF86EC76D9E324D479BB5... 남      30대                          NaN   
       FF1635DD2593732635DBA615CDCDB68FB8763D37D728D98... 남      40대                          NaN   
       FF8F1FFB4BD5D09BFF62A4A07C749E7B956AA3FC88757B8... 여      60대                          NaN   
       FFC79A4B73F8EC9BA2D1FF586F951173302BEEE799AED98... 남      40대                          NaN   
       FFCA66B0C9AD8BBE0B95471AECD627D6D52E3482EE0FCD3... 남      20대                          NaN   

      

In [178]:
combined = pd.concat([pivot_table, pivot_table2, pivot_table3, pivot_table4], axis=1, keys=['YYYYMM', 'id', 'gender', 'age'] )

combined.sort_index(level='id')
combined.to_excel('finish.xlsx')